In [1]:
sc.addPyFile("util.py")
sc.addPyFile("linear_regression.py")
sc.addPyFile("pyspark_driver.py")

import numpy as np
import linear_regression
import logging
import multiprocessing
import threading
from util import get_train_X_Y
from pyspark_driver import pyspark_cluster_test

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
ps_hosts = [ "localhost:30000" ]
worker_hosts = [ "localhost:30001", "localhost:30002"]

In [3]:
pyspark_cluster_test(sc, ps_hosts, worker_hosts)